<a href="https://colab.research.google.com/github/gerasimos-matidis/GDL_code/blob/master/plot_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount google drive and move to the project directory
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/my_icdar

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

try:
    from patchify import patchify
except:
    %pip install patchify
    from patchify import patchify

try:
    import mpl_interactions.ipyplot as iplt
except:
    %pip install mpl_interactions
    import mpl_interactions.ipyplot as iplt
    
from utils import center_crop, rebuild_from_patches

In [ ]:
# Load the input and ground truth images 
x_initial_valid = plt.imread('validation/201-INPUT.jpg')
y_initial_valid = plt.imread('validation/201-OUTPUT-GT.png')

CROP_SIZE_W = 2560 
CROP_SIZE_H = 2560

x_initial_valid = center_crop(x_initial_valid, (CROP_SIZE_H, CROP_SIZE_W))
y_initial_valid = center_crop(y_initial_valid, (CROP_SIZE_H, CROP_SIZE_W))

In [ ]:
INPUT_SHAPE = (256, 256, 3)
PATCHES_STEP = 128

input_patches = np.squeeze(patchify(x_initial_valid, INPUT_SHAPE, step=PATCHES_STEP))
ground_truth_patches = np.squeeze(patchify(y_initial_valid, INPUT_SHAPE[:-1], step=PATCHES_STEP))

x = np.reshape(input_patches, ((-1, ) + input_patches.shape[-3:]))
y = np.reshape(ground_truth_patches, ((-1, ) + ground_truth_patches.shape[-2:]))

In [ ]:
# Display a list with the available models and ask the user to choose which to use
models_path = 'trained_models/cropped_001'
models_list = os.listdir(models_path)
models = os.listdir(models_path)

predictions = []
for modelname in models:
    print(f'{modelname} is being loaded.')
    model = keras.models.load_model(os.path.join(models_path, modelname))

    splitted_modelname = modelname.split('_')[3:6] 

    d = {
        'method': splitted_modelname[0] + ' ' + splitted_modelname[1],
        'initial images': splitted_modelname[2].replace('initialimagesnumber', ''),
        'model_output': np.squeeze(model.predict(x))      
    }
    
    predictions.append(d)

In [ ]:
np.save('predictions.npy', predictions)

In [ ]:
validation_pairs = {'x': x, 'y': y}
np.save('validation_pairs.npy', validation_pairs)